<a href="https://colab.research.google.com/github/adriari4/lab-sql-query-from-table-names/blob/main/lab-sql-query-from-table-names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [3]:
# Secure way to enter your API key in Colab (hidden input)
from getpass import getpass
from openai import OpenAI

# Ask the user to type the key (it won’t be visible)
OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")

# Initialize client
client = OpenAI(api_key=OPENAI_API_KEY)

# Test the connection
print("API key loaded successfully ✅")


Enter your OpenAI API key: ··········
API key loaded successfully ✅


In [4]:
#FunctioN to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [5]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['authors', 'titles', 'sales'],
    'definition': [
        'Contains author information such as name and ID.',
        'Contains book title, price, and publisher info.',
        'Stores sales transactions for each title.'
    ]
}
df = pd.DataFrame(data)
print(df)

     table                                        definition
0  authors  Contains author information such as name and ID.
1   titles   Contains book title, price, and publisher info.
2    sales         Stores sales transactions for each title.


In [6]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [8]:
print(text_tables)

authors: Contains author information such as name and ID.
titles: Contains book title, price, and publisher info.
sales: Stores sales transactions for each title.


In [15]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [21]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="HOW MANY MONEY DO EACH AUTHOR GET") # ENTER YOUR QUERY HERE

In [22]:
print(return_OAI(pqt1))

{
    "tables": ["authors", "titles", "sales"]
}


In [19]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="") #ENTER YOUR QUERY HERE

In [20]:
print(return_OAI(pqt3))

"Can you provide me with a list of authors who have sold more than 100 copies of their books?"


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Bigger model of SQL STRUCTURE to ask more advance SQL QUERIES

In [35]:
data = {
    'table2': [
        'authors',
        'titles',
        'publishers',
        'titleauthor',
        'sales',
        'stores'
    ],
    'definition': [
        '''
        Contains author information.
        Columns:
        - au_id (Author ID, primary key)
        - au_fname (First name)
        - au_lname (Last name)
        - phone (Contact number)
        - address, city, state, zip
        - contract (Boolean: whether under contract)
        Each author may have written multiple titles.
        ''',

        '''
        Contains information about book titles.
        Columns:
        - title_id (Title ID, primary key)
        - title (Book name)
        - type (e.g., fiction, business)
        - pub_id (Publisher ID, foreign key)
        - price (Sale price)
        - advance (Advance payment to author)
        - royalty (Royalty percentage)
        - ytd_sales (Year-to-date sales)
        - pubdate (Publication date)
        Each title is published by one publisher.
        ''',

        '''
        Contains publisher information.
        Columns:
        - pub_id (Publisher ID, primary key)
        - pub_name (Publisher name)
        - city, state, country
        Publishers release multiple titles.
        ''',

        '''
        Relationship table between authors and titles.
        Columns:
        - au_id (Author ID, foreign key)
        - title_id (Title ID, foreign key)
        - royaltyper (Royalty percentage per author)
        Many-to-many relationship between authors and titles.
        ''',

        '''
        Contains sales transactions.
        Columns:
        - stor_id (Store ID, foreign key)
        - ord_num (Order number, primary key)
        - ord_date (Order date)
        - qty (Quantity sold)
        - payterms (Payment terms)
        - title_id (Title ID, foreign key)
        Each record represents the sale of a title in a store.
        ''',

        '''
        Contains store information.
        Columns:
        - stor_id (Store ID, primary key)
        - stor_name (Store name)
        - city, state, zip
        Each store may have many sales transactions.
        '''
    ]
}

df_schema = pd.DataFrame(data)


In [36]:
text_tables2 = '\n'.join([f"{row['table2']}: {row['definition']}" for index, row in df_schema.iterrows()])

In [37]:
print(text_tables2)

authors: 
        Contains author information.
        Columns:
        - au_id (Author ID, primary key)
        - au_fname (First name)
        - au_lname (Last name)
        - phone (Contact number)
        - address, city, state, zip
        - contract (Boolean: whether under contract)
        Each author may have written multiple titles.
        
titles: 
        Contains information about book titles.
        Columns:
        - title_id (Title ID, primary key)
        - title (Book name)
        - type (e.g., fiction, business)
        - pub_id (Publisher ID, foreign key)
        - price (Sale price)
        - advance (Advance payment to author)
        - royalty (Royalty percentage)
        - ytd_sales (Year-to-date sales)
        - pubdate (Publication date)
        Each title is published by one publisher.
        
publishers: 
        Contains publisher information.
        Columns:
        - pub_id (Publisher ID, primary key)
        - pub_name (Publisher name)
        - city

# ADVANCED QUERIES:

Find the total revenue (price * qty) generated by each author.

1.   Find the total revenue (price * qty) generated by each author.
2.   Show the total number of copies sold by publisher and book type
3. Display the authors who sold more than 10,000 units across all titles.
4. Find the store that sold the highest total quantity of books.

In [43]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Find the total revenue (price * qty) generated by each author.")

In [44]:
print(return_OAI(pqt1))

{
    "tables": ["authors", "titles", "sales"]
}


In [45]:
#Creating the prompt, with the user questions and the tables definitions.
pqt2 = prompt_question_tables.format(tables=text_tables, question="Show the total number of copies sold by publisher and book type")

In [46]:
print(return_OAI(pqt2))

{
    "tables": ["titles", "sales"]
}


In [47]:
#Creating the prompt, with the user questions and the tables definitions.
pqt4 = prompt_question_tables.format(tables=text_tables, question="Find the store that sold the highest total quantity of books")

In [48]:
print(return_OAI(pqt4))

{
    "tables": ["sales"]
}


In [41]:
#Creating the prompt, with the user questions and the tables definitions.
pqt3 = prompt_question_tables.format(tables=text_tables, question="Display the authors who sold more than 10,000 units across all titles") # ENTER YOUR QUERY HERE

In [42]:
print(return_OAI(pqt3))

```json
{
    "tables": ["authors", "sales"]
}
```
